In [ ]:
import json

from src.dialogue_tools import create_rag, generate_answers, generate_follow_up_questions, cross_agent_verification, aggregate_final_response, organize_into_json, display_from_json
from src.vectorstore import VectorstoreHandler
from src.models import init_llm


#####################################################################################################################
## SETUP ############################################################################################################
#####################################################################################################################
SOURCES_DIR = "sources"
JSON_PATH = "next_lvl_conversation.json"
AGGREGATION_LLM_NAME = "Llama3.2-3b"
RAG_CONFIG = {
    "book": {
        "subfolder": "book",
        "llm_name": "Llama3.2-3b",
        "emb_name": "hf-minilm-l6-v2",
        "k": 10,
    },
    "lectures": {
        "subfolder": "lectures",
        "llm_name": "Llama3.2-3b",
        "emb_name": "hf-minilm-l6-v2",
        "k": 10,
    },
}


#####################################################################################################################
#####################################################################################################################

handler = VectorstoreHandler(SOURCES_DIR, force_rebuild=False)
rags = {rag_name: create_rag(SOURCES_DIR, config, handler) for rag_name, config in RAG_CONFIG.items()}

aggregation_llm = init_llm(AGGREGATION_LLM_NAME)


Loading existing vectorstore for sources/book...
Loading existing vectorstore for sources/lectures...


In [4]:
original_prompt = "Give a point process model that is similar to the Ising model and explain what the similarities are"

# Step 1: Initial Question to All Agents
initial_responses = generate_answers(original_prompt, rags)

# Step 2: Follow-Up Questions from Aggregation LLM
agent_answers = {k: v["answer"] for k, v in initial_responses.items()}
follow_up_questions = generate_follow_up_questions(aggregation_llm, original_prompt, agent_answers)

# Step 3: Follow-Up Answers from Original Agents
follow_up_responses = {
    agent_name: generate_answers(question, {agent_name: rags[agent_name]})[agent_name]
    for agent_name, question in follow_up_questions.items()
}

# Step 4: Cross-Agent Verification
cross_agent_responses = cross_agent_verification(rags, follow_up_responses, original_prompt, follow_up_questions)
print(cross_agent_responses)
# Step 5: Final Aggregation
all_responses = {
    agent_name: {
        "initial": initial_responses[agent_name]["answer"],
        "follow_up": follow_up_responses[agent_name]["answer"],
    }
    for agent_name in rags.keys()
}
final_answer = aggregate_final_response(
    aggregation_llm, all_responses, original_prompt
)

# Step 6: Organize and Display
conversation_json = organize_into_json(
    original_prompt, initial_responses, follow_up_questions, follow_up_responses, final_answer
)
conversation_json["cross_agent_responses"] = {
    target_agent: {
        verifier: response.get("answer", "No answer provided")
        for verifier, response in cross_agent_responses.get(target_agent, {}).items()
    }
    for target_agent in cross_agent_responses.keys()
}
display_from_json(conversation_json)

# Save JSON
json.dump(conversation_json, open(JSON_PATH, "w"), indent=4)


Querying all RAGs for prompt: 'Give a point process model that is similar to the Ising model and explain what the similarities are'
--- Querying RAG: book ---
--- Querying RAG: lectures ---

Querying all RAGs for prompt: 'The original question was:
Give a point process model that is similar to the Ising model and explain what the similarities are

Here's the follow-up question:
Can you elaborate on how the Ising lattice model for point processes captures the underlying dynamics of point process behavior, such as cluster formation and spatial clustering, which are common phenomena observed in natural systems like percolation and branching networks?'
--- Querying RAG: book ---

Querying all RAGs for prompt: 'The original question was:
Give a point process model that is similar to the Ising model and explain what the similarities are

Here's the follow-up question:
What is the impact of the parameter γ on the mean-field version of the Potts model, and how does it compare to the Ising mod